In [1]:
#importing libraries
import numpy as np
import pandas as pd
from scipy.signal import filtfilt
from scipy.fft import fft, fftfreq, fftshift
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from scipy import stats
import math
import scipy
import pywt



# signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz

from scipy.signal import butter, sosfilt, sosfilt_zi, sosfiltfilt, lfilter, lfilter_zi, filtfilt, sosfreqz, resample
#from utils import hamilton_detector, christov_detector, findpeaks, engzee_detector
#from ecgdetectors import Detectors, MWA, panPeakDetect, searchBack

# misc
import warnings



In [ ]:
import os
print(os.listdir("C:/Users/user/Desktop/ECG classification/dataset"))

mit_test_data = pd.read_csv("C:/Users/user/Desktop/ECG classification/dataset/mitbih_test.csv", header=None)
mit_train_data = pd.read_csv("C:/Users/user/Desktop/ECG classification/dataset/mitbih_train.csv", header=None)

print("MIT test dataset")
print(mit_test_data.info())
print("MIT train dataset")
print(mit_train_data.info())

mit_train_data

['mitbih_test.csv', 'mitbih_train.csv']


In [ ]:
# take a random distribution
sample = mit_test_data.sample(25)

# remove the target column
sampleX = sample.iloc[:,sample.columns != 187]


plt.style.use('classic')

# plt samples
for index, row in sampleX.iterrows():
    plt.plot(np.array(range(0, 187)) ,row)

plt.xlabel("time")
plt.ylabel("magnitude")
plt.title("heartbeat reccording \nrandom sample")

plt.show()

plt.style.use("ggplot")

plt.title("Number of record in each category")

plt.hist(sample.iloc[:,sample.columns == 187].transpose())
plt.show()

In [ ]:
# plot ECG data belonging to each class label
label = ["Non-ecotic beats","Supraventricular ectopic beats","Ventricular ectopic beats",
         "Fusion Beats","Unknown Beats"]
color = ['blue','red','green','orange','purple','black']
fig, ax = plt.subplots(5, 1, sharex=True, sharey=True,figsize=(20,12))
for i, row in enumerate(ax):
    row.plot((mit_train_data[mit_train_data[187] == i].iloc[0])[:-1], label=label[i], color=color[i])
    row.legend(fontsize=12)
fig.text(0.5, 0.04, 'Amplitude', va='center', ha='center', fontsize=15)
fig.text(0.04, 0.5, 'Time (ms)', va='center', ha='center', rotation='vertical', fontsize=15)

plt.show()


In [ ]:
#imbalance of samples of each class
print("Train data")
print("Type\tCount")
print((mit_train_data[187]).value_counts())
print("-------------------------")
print("Test data")
print("Type\tCount")
print((mit_test_data[187]).value_counts())

In [ ]:
#plotting the circle diagram giving the percentage of each class
plt.figure(figsize=(10,7))
equilibre=mit_train_data[187].value_counts()
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(equilibre, labels=['n','q','v','s','f'], colors=['red','green','blue','skyblue','orange'],autopct='%1.1f%%')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

In [ ]:
#resampling the  train data 
from sklearn.utils import resample
df_1=mit_train_data[mit_train_data[187]==1]
df_2=mit_train_data[mit_train_data[187]==2]
df_3=mit_train_data[mit_train_data[187]==3]
df_4=mit_train_data[mit_train_data[187]==4]
df_0=(mit_train_data[mit_train_data[187]==0]).sample(n=20000,random_state=42)

df_1_upsample=resample(df_1,replace=True,n_samples=20000,random_state=123)
df_2_upsample=resample(df_2,replace=True,n_samples=20000,random_state=124)
df_3_upsample=resample(df_3,replace=True,n_samples=20000,random_state=125)
df_4_upsample=resample(df_4,replace=True,n_samples=20000,random_state=126)

mit_train_data=pd.concat([df_0,df_1_upsample,df_2_upsample,df_3_upsample,df_4_upsample])



In [ ]:
#verifying the equal between the classes
equilibre=mit_train_data[187].value_counts()
print(equilibre)

mit_train_data

In [ ]:
#plotting the diagram cercle after resample (train data)
plt.figure(figsize=(10,7))
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(equilibre, labels=['n','q','v','s','f'], colors=['red','green','blue','skyblue','orange'],autopct='%1.1f%%')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

In [ ]:
#extracting each row from train data and adding it within list into another list
list_samples_normal=[]
for i in range(0,100000) :
    y=mit_train_data.iloc[i]
    s=list(y)
    list_samples_normal.append(s[:-1])


len(list_samples_normal)

In [ ]:
#building list time with plotting a sample
list_time=[i*(1/125) for i in range(0,187)] #time axes
plt.plot(list_time,list_samples_normal[2])

In [ ]:
#algorithm that capable of extracting all features wihthin their correspondance class of the train data
AllSignals_train=[]
Class_train=[]
for j in range(0,7000):
    Class_train.append(mit_train_data.iloc[j][187])
    percentile_list = pd.DataFrame({'samples': list_samples_normal[j]})
    features=percentile_list.describe()
    list_samples_test=[]
    for i in range(1,7) :
        if i!=2:
            n=features.iloc[i]
            m=list(n)[0]
            list_samples_test.append(m)
    AllSignals_train.append(list_samples_test)
#np.save("signals",AllSignals)
#L=np.load("signals")

In [ ]:
v=0
for e in range(0,7000) :
    if Class_train[e]==0 :
        v+=1

In [ ]:
k=0
for i in AllSignals:
    if i[3]!=0:
        k+=1

In [ ]:
v

In [ ]:
#verifying the equal between the classes
equilibre_1=mit_test_data[187].value_counts()
print(equilibre_1)

mit_test_data

In [ ]:
#plotting the circle diagram giving the percentage of each class (test data)
plt.figure(figsize=(10,7))
equilibre_1=mit_test_data[187].value_counts()
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(equilibre_1, labels=['n','q','v','s','f'], colors=['red','green','blue','skyblue','orange'],autopct='%1.1f%%')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

In [ ]:
#resampling the  test data 
from sklearn.utils import resample
df_11=mit_test_data[mit_test_data[187]==1]
df_22=mit_test_data[mit_test_data[187]==2]
df_33=mit_test_data[mit_test_data[187]==3]
df_44=mit_test_data[mit_test_data[187]==4]
df_00=(mit_test_data[mit_test_data[187]==0]).sample(n=10000,random_state=42)

df_11_upsample=resample(df_11,replace=True,n_samples=10000,random_state=123)
df_22_upsample=resample(df_22,replace=True,n_samples=10000,random_state=124)
df_33_upsample=resample(df_33,replace=True,n_samples=10000,random_state=125)
df_44_upsample=resample(df_44,replace=True,n_samples=10000,random_state=126)

mit_test_data=pd.concat([df_00,df_11_upsample,df_22_upsample,df_33_upsample,df_44_upsample])

In [ ]:
#verifying the equal between the classes (test data)
equilibre_2=mit_test_data[187].value_counts()
print(equilibre_2)

mit_test_data

In [ ]:
#plotting the circle diagram giving the percentage of each class (test data)
plt.figure(figsize=(10,7))
equilibre_3=mit_test_data[187].value_counts()
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(equilibre_3, labels=['n','q','v','s','f'], colors=['red','green','blue','skyblue','orange'],autopct='%1.1f%%')
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

In [ ]:
#extracting each row from test data and adding it within list into another list
list_samples_normal_test=[]
for i in range(0,50000) :
    yy=mit_test_data.iloc[i]
    ss=list(yy)
    list_samples_normal_test.append(s[:-1])


len(list_samples_normal_test)

In [ ]:
#building list time with plotting a sample
list_time_test=[i*(1/125) for i in range(0,187)] #time axes
plt.plot(list_time,list_samples_normal_test[2])

In [ ]:
#algorithm that capable of extracting all features wihthin their correspondance class of the train data
AllSignals_test=[]
Class_test=[]
for j in range(0,7000):
    Class_test.append(mit_test_data.iloc[j][187])
    percentile_list_test = pd.DataFrame({'samples': list_samples_normal_test[j]})
    features_test=percentile_list_test.describe()
    list_samples_test_t=[]
    for i in range(1,7) :
        if i!=2:
            n_t=features_test.iloc[i]
            m_t=list(n_t)[0]
            list_samples_test_t.append(m_t)
    AllSignals_test.append(list_samples_test_t)
#np.save("signals",AllSignals)
#L=np.load("signals")

In [ ]:
Class_test

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy


In [ ]:
# Create the model
model = tf.keras.Sequential([
 Dense(units = 5, activation = 'linear', input_shape=[5]),
 Dense(units = 64, activation = 'relu'),
 Dense(units = 64, activation = 'relu'),
 Dense(units = 1, activation = 'linear')
])
#model.compile(loss='mse', optimizer="adam")
# Display the mode
model.summary()

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.0065, decay_steps=10000, decay_rate=0.085)
    #Optimizers List
SGGD = keras.optimizers.SGD(learning_rate=lr_schedule)
RMSPROOP = keras.optimizers.RMSprop(learning_rate=lr_schedule)
ADAAM = Adam(lr_schedule)
model.compile(optimizer=ADAAM, loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)
history=model.fit(AllSignals_train,Class_train,validation_data=(AllSignals_test, Class_test),batch_size=1,epochs=10, verbose=2,callbacks=[stop_early])

Epoch 1/10
7000/7000 - 57s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - 57s/epoch - 8ms/step
Epoch 2/10
7000/7000 - 68s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - 68s/epoch - 10ms/step
Epoch 3/10
7000/7000 - 70s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - 70s/epoch - 10ms/step
Epoch 4/10
7000/7000 - 55s - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000 - 55s/epoch - 8ms/step
Epoch 5/10


KeyboardInterrupt: 

In [19]:
history.history


{'loss': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'accuracy': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 'val_loss': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'val_accuracy': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}

In [4]:
import pip 
! pip install --user tensorflow

^C
